In [1]:
import json
from pyserini.search import FaissSearcher, LuceneSearcher,LuceneImpactSearcher
from tqdm import tqdm
import numpy as np
import random

with open('dl19_hy', 'r') as file:
    dl19_hy = json.load(file)

with open('dl19_topics', 'r') as file:
    dl19_topics = json.load(file)

In [2]:
#0.0 bm25

searcher = LuceneSearcher.from_prebuilt_index('msmarco-v1-passage')

with open('dl19_bm25', 'w')  as f:
    for i in range(len(dl19_topics)):
        qid=dl19_topics[i][0]
        question=dl19_topics[i][1]
        hits = searcher.search(question, k=1000)
        rank = 0
        for hit in hits:
            rank += 1
            f.write(f'{qid} Q0 {hit.docid} {rank} {hit.score} rank\n')

!python -m pyserini.eval.trec_eval -c -l 2 -m map dl19-passage dl19_bm25
!python -m pyserini.eval.trec_eval -c -m ndcg_cut.10 dl19-passage dl19_bm25
!python -m pyserini.eval.trec_eval -c -l 2 -m recall.1000 dl19-passage dl19_bm25

Dec 15, 2024 9:35:36 PM org.apache.lucene.store.MemorySegmentIndexInputProvider <init>
INFO: Using MemorySegmentIndexInput with Java 21; to disable start with -Dorg.apache.lucene.store.MMapDirectory.enableMemorySegments=false


/root/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar already exists!
Skipping download.
Running command: ['java', '-jar', '/root/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar', '-c', '-l', '2', '-m', 'map', '/root/.cache/pyserini/topics-and-qrels/qrels.dl19-passage.txt', 'dl19_bm25']
Results:
map                   	all	0.3013
/root/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar already exists!
Skipping download.
Running command: ['java', '-jar', '/root/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar', '-c', '-m', 'ndcg_cut.10', '/root/.cache/pyserini/topics-and-qrels/qrels.dl19-passage.txt', 'dl19_bm25']
Results:
ndcg_cut_10           	all	0.5058
/root/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar already exists!
Skipping download.
Running command: ['java', '-jar', '/root/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar', '-c', '-l', '2', '-m', 'recall.1000', '/root/.cache/pyserini/topics-and

In [3]:
#0.1 Bm25+Exp4Fuse 
searcher = LuceneSearcher.from_prebuilt_index('msmarco-v1-passage')

with open('dl19_bm25_hy', 'w')  as f:
    for i in range(len(dl19_hy)):
        qid=dl19_hy[i][0]
        question=dl19_hy[i][1]+'.'
        question=question*5
        question=question+dl19_hy[i][2]
        hits = searcher.search(question, k=1000)
        rank = 0
        for hit in hits:
            rank += 1
            f.write(f'{qid} Q0 {hit.docid} {rank} {hit.score} rank\n')


with open('dl19_bm25_hy', 'r') as file:
    hy1 = file.readlines()      
hy=[[int(x.split()[0]),x.split()[1],int(x.split()[2]),int(x.split()[3]),x.split()[4],x.split()[5]] for x in hy1]

with open('dl19_bm25', 'r') as file:
    ori1 = file.readlines()
ori=[[int(x.split()[0]),x.split()[1],int(x.split()[2]),int(x.split()[3]),x.split()[4],x.split()[5]] for x in ori1]

ratio=60

with open('dl19_bm25_Exp4Fuse', 'w')  as f:
    for i in range(len(dl19_topics)):
        dictTem={}
        for j in range(len(hy)):
            if dl19_topics[i][0]==hy[j][0]:
                try:
                    dictTem[hy[j][2]]['score1']=1/(ratio+hy[j][3])
                except KeyError:
                    dictTem[hy[j][2]]={}
                    dictTem[hy[j][2]]['score1']=1/(ratio+hy[j][3])              
        for j in range(len(ori)):
            if dl19_topics[i][0]==ori[j][0]:
                try:
                    dictTem[ori[j][2]]['score3']=1/(ratio+ori[j][3])
                except KeyError:
                    dictTem[ori[j][2]]={}
                    dictTem[ori[j][2]]['score3']=1/(ratio+ori[j][3])  
        for passage in dictTem.keys():
            num=len(dictTem[passage])/10+1
            score_sum=0
            for scores in dictTem[passage].keys():
                score_sum+=dictTem[passage][scores]
            dictTem[passage]=num*score_sum
        rank=0
        for key, value in sorted(dictTem.items(), key=lambda item: item[1],reverse=True):   
            rank+=1
            if rank<=1000:
                passageinfo=[dl19_hy[i][0],'Q0',key,rank,value,'rank']
                topicnum=dl19_topics[i][0]
                f.write(f'{topicnum} Q0 {key} {rank} {value} rank\n')

!python -m pyserini.eval.trec_eval -c -l 2 -m map dl19-passage dl19_bm25_Exp4Fuse
!python -m pyserini.eval.trec_eval -c -m ndcg_cut.10 dl19-passage dl19_bm25_Exp4Fuse
!python -m pyserini.eval.trec_eval -c -l 2 -m recall.1000 dl19-passage dl19_bm25_Exp4Fuse

/root/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar already exists!
Skipping download.
Running command: ['java', '-jar', '/root/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar', '-c', '-l', '2', '-m', 'map', '/root/.cache/pyserini/topics-and-qrels/qrels.dl19-passage.txt', 'dl19_bm25_Exp4Fuse']
Results:
map                   	all	0.3871
/root/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar already exists!
Skipping download.
Running command: ['java', '-jar', '/root/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar', '-c', '-m', 'ndcg_cut.10', '/root/.cache/pyserini/topics-and-qrels/qrels.dl19-passage.txt', 'dl19_bm25_Exp4Fuse']
Results:
ndcg_cut_10           	all	0.6199
/root/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar already exists!
Skipping download.
Running command: ['java', '-jar', '/root/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar', '-c', '-l', '2', '-m', 'recall.1000', '/root/.cache/p

In [4]:
#1.0 docT5query

searcher = LuceneSearcher('autodl-tmp/d2q-t5')

with open('dl19_d2q', 'w')  as f:
    for i in range(len(dl19_topics)):
        qid=dl19_topics[i][0]
        question=dl19_topics[i][1]
        hits = searcher.search(question, k=1000)
        rank = 0
        for hit in hits:
            rank += 1
            f.write(f'{qid} Q0 {hit.docid} {rank} {hit.score} rank\n')

!python -m pyserini.eval.trec_eval -c -l 2 -m map dl19-passage dl19_d2q
!python -m pyserini.eval.trec_eval -c -m ndcg_cut.10 dl19-passage dl19_d2q
!python -m pyserini.eval.trec_eval -c -l 2 -m recall.1000 dl19-passage dl19_d2q

/root/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar already exists!
Skipping download.
Running command: ['java', '-jar', '/root/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar', '-c', '-l', '2', '-m', 'map', '/root/.cache/pyserini/topics-and-qrels/qrels.dl19-passage.txt', 'dl19_d2q']
Results:
map                   	all	0.4034
/root/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar already exists!
Skipping download.
Running command: ['java', '-jar', '/root/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar', '-c', '-m', 'ndcg_cut.10', '/root/.cache/pyserini/topics-and-qrels/qrels.dl19-passage.txt', 'dl19_d2q']
Results:
ndcg_cut_10           	all	0.6417
/root/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar already exists!
Skipping download.
Running command: ['java', '-jar', '/root/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar', '-c', '-l', '2', '-m', 'recall.1000', '/root/.cache/pyserini/topics-and-q

In [5]:
#1.1 docT5query+Exp4Fuse
searcher = LuceneSearcher('autodl-tmp/d2q-t5')

with open('dl19_d2q_hy', 'w')  as f:
    for i in range(len(dl19_hy)):
        qid=dl19_hy[i][0]
        question=dl19_hy[i][1]+'.'
        question=question*5
        question=question+dl19_hy[i][2]
        hits = searcher.search(question, k=1000)
        rank = 0
        for hit in hits:
            rank += 1
            f.write(f'{qid} Q0 {hit.docid} {rank} {hit.score} rank\n')
    
with open('dl19_d2q_hy', 'r') as file:
    hy1 = file.readlines()
hy=[[int(x.split()[0]),x.split()[1],int(x.split()[2]),int(x.split()[3]),x.split()[4],x.split()[5]] for x in hy1]


with open('dl19_d2q', 'r') as file:
    ori1 = file.readlines()
ori=[[int(x.split()[0]),x.split()[1],int(x.split()[2]),int(x.split()[3]),x.split()[4],x.split()[5]] for x in ori1]


with open('dl19_d2q_Exp4Fuse', 'w')  as f:
    for i in range(len(dl19_topics)):
        dictTem={}
        for j in range(len(hy)):
            if dl19_topics[i][0]==hy[j][0]:
                try:
                    dictTem[hy[j][2]]['score1']=1/(ratio+hy[j][3])
                except KeyError:
                    dictTem[hy[j][2]]={}
                    dictTem[hy[j][2]]['score1']=1/(ratio+hy[j][3])               
        for j in range(len(ori)):
            if dl19_topics[i][0]==ori[j][0]:
                try:
                    dictTem[ori[j][2]]['score3']=1/(ratio+ori[j][3])
                except KeyError:
                    dictTem[ori[j][2]]={}
                    dictTem[ori[j][2]]['score3']=1/(ratio+ori[j][3])  
        for passage in dictTem.keys():
            num=len(dictTem[passage])/10+1
            score_sum=0
            for scores in dictTem[passage].keys():
                score_sum+=dictTem[passage][scores]
            dictTem[passage]=num*score_sum
        rank=0
        for key, value in sorted(dictTem.items(), key=lambda item: item[1],reverse=True):   
            rank+=1
            if rank<=1000:
                passageinfo=[dl19_hy[i][0],'Q0',key,rank,value,'rank']
                topicnum=dl19_topics[i][0]
                f.write(f'{topicnum} Q0 {key} {rank} {value} rank\n')

!python -m pyserini.eval.trec_eval -c -l 2 -m map dl19-passage dl19_d2q_Exp4Fuse
!python -m pyserini.eval.trec_eval -c -m ndcg_cut.10 dl19-passage dl19_d2q_Exp4Fuse
!python -m pyserini.eval.trec_eval -c -l 2 -m recall.1000 dl19-passage dl19_d2q_Exp4Fuse
   

/root/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar already exists!
Skipping download.
Running command: ['java', '-jar', '/root/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar', '-c', '-l', '2', '-m', 'map', '/root/.cache/pyserini/topics-and-qrels/qrels.dl19-passage.txt', 'dl19_d2q_Exp4Fuse']
Results:
map                   	all	0.4749
/root/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar already exists!
Skipping download.
Running command: ['java', '-jar', '/root/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar', '-c', '-m', 'ndcg_cut.10', '/root/.cache/pyserini/topics-and-qrels/qrels.dl19-passage.txt', 'dl19_d2q_Exp4Fuse']
Results:
ndcg_cut_10           	all	0.6854
/root/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar already exists!
Skipping download.
Running command: ['java', '-jar', '/root/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar', '-c', '-l', '2', '-m', 'recall.1000', '/root/.cache/pys

In [6]:
#2.0 unicoil

searcher = LuceneImpactSearcher('autodl-tmp/msmarco-v1-passage.unicoil','autodl-tmp/unicoil-msmarco-passage')

with open('dl19_unicoil', 'w')  as f:
    for i in range(len(dl19_topics)):
        qid=dl19_topics[i][0]
        question=dl19_topics[i][1]
        hits = searcher.search(question, k=1000)
        rank = 0
        for hit in hits:
            rank += 1
            f.write(f'{qid} Q0 {hit.docid} {rank} {hit.score} rank\n')

!python -m pyserini.eval.trec_eval -c -l 2 -m map dl19-passage dl19_unicoil
!python -m pyserini.eval.trec_eval -c -m ndcg_cut.10 dl19-passage dl19_unicoil
!python -m pyserini.eval.trec_eval -c -l 2 -m recall.1000 dl19-passage dl19_unicoil

/root/miniconda3/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
/root/miniconda3/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


/root/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar already exists!
Skipping download.
Running command: ['java', '-jar', '/root/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar', '-c', '-l', '2', '-m', 'map', '/root/.cache/pyserini/topics-and-qrels/qrels.dl19-passage.txt', 'dl19_unicoil']
Results:
map                   	all	0.4612
/root/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar already exists!
Skipping download.
Running command: ['java', '-jar', '/root/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar', '-c', '-m', 'ndcg_cut.10', '/root/.cache/pyserini/topics-and-qrels/qrels.dl19-passage.txt', 'dl19_unicoil']
Results:
ndcg_cut_10           	all	0.7024
/root/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar already exists!
Skipping download.
Running command: ['java', '-jar', '/root/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar', '-c', '-l', '2', '-m', 'recall.1000', '/root/.cache/pyserini/topi

In [7]:
#2.1 unicoil +Exp4Fuse
searcher = LuceneImpactSearcher('autodl-tmp/msmarco-v1-passage.unicoil','autodl-tmp/unicoil-msmarco-passage')

with open('dl19_unicoil_hy', 'w')  as f:
    for i in range(len(dl19_hy)):
        qid=dl19_hy[i][0]
        question=dl19_hy[i][1]+'.'
        question=question*5
        question=question+dl19_hy[i][2]
        hits = searcher.search(question, k=1000)
        rank = 0
        for hit in hits:
            rank += 1
            f.write(f'{qid} Q0 {hit.docid} {rank} {hit.score} rank\n')

with open('dl19_unicoil_hy', 'r') as file:
    hy1 = file.readlines()      
hy=[[int(x.split()[0]),x.split()[1],int(x.split()[2]),int(x.split()[3]),x.split()[4],x.split()[5]] for x in hy1]

with open('dl19_unicoil', 'r') as file:
    ori1 = file.readlines()
ori=[[int(x.split()[0]),x.split()[1],int(x.split()[2]),int(x.split()[3]),x.split()[4],x.split()[5]] for x in ori1]


with open('dl19_unicoil_Exp4Fuse', 'w')  as f:
    for i in range(len(dl19_topics)):
        dictTem={}
        for j in range(len(hy)):
            if dl19_topics[i][0]==hy[j][0]:
                try:
                    dictTem[hy[j][2]]['score1']=1/(ratio+hy[j][3])
                except KeyError:
                    dictTem[hy[j][2]]={}
                    dictTem[hy[j][2]]['score1']=1/(ratio+hy[j][3])             
        for j in range(len(ori)):
            if dl19_topics[i][0]==ori[j][0]:
                try:
                    dictTem[ori[j][2]]['score3']=1/(ratio+ori[j][3])
                except KeyError:
                    dictTem[ori[j][2]]={}
                    dictTem[ori[j][2]]['score3']=1/(ratio+ori[j][3])  
        for passage in dictTem.keys():
            num=len(dictTem[passage])/10+1
            score_sum=0
            for scores in dictTem[passage].keys():
                score_sum+=dictTem[passage][scores]
            dictTem[passage]=num*score_sum
        rank=0
        for key, value in sorted(dictTem.items(), key=lambda item: item[1],reverse=True):   
            rank+=1
            if rank<=1000:
                topicnum=dl19_topics[i][0]
                f.write(f'{topicnum} Q0 {key} {rank} {value} rank\n')

!python -m pyserini.eval.trec_eval -c -l 2 -m map dl19-passage dl19_unicoil_Exp4Fuse
!python -m pyserini.eval.trec_eval -c -m ndcg_cut.10 dl19-passage dl19_unicoil_Exp4Fuse
!python -m pyserini.eval.trec_eval -c -l 2 -m recall.1000 dl19-passage dl19_unicoil_Exp4Fuse
   


/root/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar already exists!
Skipping download.
Running command: ['java', '-jar', '/root/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar', '-c', '-l', '2', '-m', 'map', '/root/.cache/pyserini/topics-and-qrels/qrels.dl19-passage.txt', 'dl19_unicoil_Exp4Fuse']
Results:
map                   	all	0.5007
/root/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar already exists!
Skipping download.
Running command: ['java', '-jar', '/root/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar', '-c', '-m', 'ndcg_cut.10', '/root/.cache/pyserini/topics-and-qrels/qrels.dl19-passage.txt', 'dl19_unicoil_Exp4Fuse']
Results:
ndcg_cut_10           	all	0.7352
/root/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar already exists!
Skipping download.
Running command: ['java', '-jar', '/root/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar', '-c', '-l', '2', '-m', 'recall.1000', '/root/.c

In [2]:
#3.0 slimrpp

searcher = LuceneImpactSearcher('autodl-tmp/msmarco-v1-passage.slimr-pp','autodl-tmp/slimr-pp-msmarco-passage')

with open('dl19_slimrpp', 'w')  as f:
    for i in range(len(dl19_topics)):
        qid=dl19_topics[i][0]
        question=dl19_topics[i][1]
        hits = searcher.search(question, k=1000)
        rank = 0
        for hit in hits:
            rank += 1
            f.write(f'{qid} Q0 {hit.docid} {rank} {hit.score} rank\n')

!python -m pyserini.eval.trec_eval -c -l 2 -m map dl19-passage dl19_slimrpp
!python -m pyserini.eval.trec_eval -c -m ndcg_cut.10 dl19-passage dl19_slimrpp
!python -m pyserini.eval.trec_eval -c -l 2 -m recall.1000 dl19-passage dl19_slimrpp

Dec 15, 2024 9:43:40 PM org.apache.lucene.store.MemorySegmentIndexInputProvider <init>
INFO: Using MemorySegmentIndexInput with Java 21; to disable start with -Dorg.apache.lucene.store.MMapDirectory.enableMemorySegments=false
/root/miniconda3/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
/root/miniconda3/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/318

/root/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar already exists!
Skipping download.
Running command: ['java', '-jar', '/root/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar', '-c', '-l', '2', '-m', 'map', '/root/.cache/pyserini/topics-and-qrels/qrels.dl19-passage.txt', 'dl19_slimrpp']
Results:
map                   	all	0.4828


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


/root/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar already exists!
Skipping download.
Running command: ['java', '-jar', '/root/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar', '-c', '-m', 'ndcg_cut.10', '/root/.cache/pyserini/topics-and-qrels/qrels.dl19-passage.txt', 'dl19_slimrpp']
Results:
ndcg_cut_10           	all	0.7245


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


/root/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar already exists!
Skipping download.
Running command: ['java', '-jar', '/root/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar', '-c', '-l', '2', '-m', 'recall.1000', '/root/.cache/pyserini/topics-and-qrels/qrels.dl19-passage.txt', 'dl19_slimrpp']
Results:
recall_1000           	all	0.8681


In [2]:
#3.1 slimrpp+Exp4Fuse

searcher = LuceneImpactSearcher('autodl-tmp/msmarco-v1-passage.slimr-pp','autodl-tmp/slimr-pp-msmarco-passage')

with open('dl19_slimrpp_hy', 'w')  as f:
    for i in range(len(dl19_hy)):
        qid=dl19_hy[i][0]
        question=dl19_hy[i][1]+'.'
        question=question*5
        question=question+dl19_hy[i][2]
        hits = searcher.search(question, k=1000)
        rank = 0
        for hit in hits:
            rank += 1
            f.write(f'{qid} Q0 {hit.docid} {rank} {hit.score} rank\n')

with open('dl19_slimrpp_hy', 'r') as file:
    hy1 = file.readlines()        
hy=[[int(x.split()[0]),x.split()[1],int(x.split()[2]),int(x.split()[3]),x.split()[4],x.split()[5]] for x in hy1]

with open('dl19_slimrpp', 'r') as file:
    ori1 = file.readlines()
ori=[[int(x.split()[0]),x.split()[1],int(x.split()[2]),int(x.split()[3]),x.split()[4],x.split()[5]] for x in ori1]

ratio=60

with open('dl19_slimrpp_Exp4Fuse', 'w')  as f:
    for i in range(len(dl19_topics)):
        dictTem={}
        for j in range(1000*i,1000*(i+1)):
            if dl19_topics[i][0]==hy[j][0]:
                try:
                    dictTem[hy[j][2]]['score1']=1/(ratio+hy[j][3])
                except KeyError:
                    dictTem[hy[j][2]]={}
                    dictTem[hy[j][2]]['score1']=1/(ratio+hy[j][3])              
        for j in range(len(ori)):
            if dl19_topics[i][0]==ori[j][0]:
                try:
                    dictTem[ori[j][2]]['score3']=1/(ratio+ori[j][3])
                except KeyError:
                    dictTem[ori[j][2]]={}
                    dictTem[ori[j][2]]['score3']=1/(ratio+ori[j][3])  
        for passage in dictTem.keys():
            num=len(dictTem[passage])/10+1
            score_sum=0
            for scores in dictTem[passage].keys():
                score_sum+=dictTem[passage][scores]
            dictTem[passage]=num*score_sum
        rank=0
        for key, value in sorted(dictTem.items(), key=lambda item: item[1],reverse=True):   
            rank+=1
            if rank<=1000:
                topicnum=dl19_topics[i][0]
                f.write(f'{topicnum} Q0 {key} {rank} {value} rank\n')

!python -m pyserini.eval.trec_eval -c -l 2 -m map dl19-passage dl19_slimrpp_Exp4Fuse
!python -m pyserini.eval.trec_eval -c -m ndcg_cut.10 dl19-passage dl19_slimrpp_Exp4Fuse
!python -m pyserini.eval.trec_eval -c -l 2 -m recall.1000 dl19-passage dl19_slimrpp_Exp4Fuse

Dec 15, 2024 9:49:21 PM org.apache.lucene.store.MemorySegmentIndexInputProvider <init>
INFO: Using MemorySegmentIndexInput with Java 21; to disable start with -Dorg.apache.lucene.store.MMapDirectory.enableMemorySegments=false
/root/miniconda3/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
/root/miniconda3/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/318

/root/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar already exists!
Skipping download.
Running command: ['java', '-jar', '/root/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar', '-c', '-l', '2', '-m', 'map', '/root/.cache/pyserini/topics-and-qrels/qrels.dl19-passage.txt', 'dl19_slimrpp_Exp4Fuse']
Results:
map                   	all	0.5047


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


/root/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar already exists!
Skipping download.
Running command: ['java', '-jar', '/root/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar', '-c', '-m', 'ndcg_cut.10', '/root/.cache/pyserini/topics-and-qrels/qrels.dl19-passage.txt', 'dl19_slimrpp_Exp4Fuse']
Results:
ndcg_cut_10           	all	0.7638


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


/root/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar already exists!
Skipping download.
Running command: ['java', '-jar', '/root/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar', '-c', '-l', '2', '-m', 'recall.1000', '/root/.cache/pyserini/topics-and-qrels/qrels.dl19-passage.txt', 'dl19_slimrpp_Exp4Fuse']
Results:
recall_1000           	all	0.8717


In [8]:
#4.0 splade_ed

searcher = LuceneImpactSearcher('autodl-tmp/msmarco-v1-passage.splade-pp-ed','autodl-tmp/splade-cocondenser-ensembledistil')

with open('dl19_splade_ed', 'w')  as f:
    for i in range(len(dl19_topics)):
        qid=dl19_topics[i][0]
        question=dl19_topics[i][1]
        hits = searcher.search(question, k=1000)
        rank = 0
        for hit in hits:
            rank += 1
            f.write(f'{qid} Q0 {hit.docid} {rank} {hit.score} rank\n')

!python -m pyserini.eval.trec_eval -c -l 2 -m map dl19-passage dl19_splade_ed
!python -m pyserini.eval.trec_eval -c -m ndcg_cut.10 dl19-passage dl19_splade_ed
!python -m pyserini.eval.trec_eval -c -l 2 -m recall.1000 dl19-passage dl19_splade_ed

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


/root/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar already exists!
Skipping download.
Running command: ['java', '-jar', '/root/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar', '-c', '-l', '2', '-m', 'map', '/root/.cache/pyserini/topics-and-qrels/qrels.dl19-passage.txt', 'dl19_splade_ed']
Results:
map                   	all	0.5050


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


/root/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar already exists!
Skipping download.
Running command: ['java', '-jar', '/root/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar', '-c', '-m', 'ndcg_cut.10', '/root/.cache/pyserini/topics-and-qrels/qrels.dl19-passage.txt', 'dl19_splade_ed']
Results:
ndcg_cut_10           	all	0.7308


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


/root/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar already exists!
Skipping download.
Running command: ['java', '-jar', '/root/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar', '-c', '-l', '2', '-m', 'recall.1000', '/root/.cache/pyserini/topics-and-qrels/qrels.dl19-passage.txt', 'dl19_splade_ed']
Results:
recall_1000           	all	0.8728


In [9]:
#4.1 splade_ed+Exp4Fuse

searcher = LuceneImpactSearcher('autodl-tmp/msmarco-v1-passage.splade-pp-ed','autodl-tmp/splade-cocondenser-ensembledistil')


with open('dl19_splade_ed_hy', 'w')  as f:
    for i in range(len(dl19_hy)):
        qid=dl19_hy[i][0]
        question=dl19_hy[i][1]+'.'
        question=question*5
        question=question+dl19_hy[i][2]
        hits = searcher.search(question, k=1000)
        rank = 0
        for hit in hits:
            rank += 1
            f.write(f'{qid} Q0 {hit.docid} {rank} {hit.score} rank\n')

    
with open('dl19_splade_ed_hy', 'r') as file:
    hy1 = file.readlines()       
hy=[[int(x.split()[0]),x.split()[1],int(x.split()[2]),int(x.split()[3]),x.split()[4],x.split()[5]] for x in hy1]


with open('dl19_splade_ed', 'r') as file:
    ori1 = file.readlines()
ori=[[int(x.split()[0]),x.split()[1],int(x.split()[2]),int(x.split()[3]),x.split()[4],x.split()[5]] for x in ori1]

with open('dl19_splade_ed_Exp4Fuse', 'w')  as f:
    for i in range(len(dl19_topics)):
        dictTem={}
        for j in range(len(hy)):
            if dl19_topics[i][0]==hy[j][0]:
                try:
                    dictTem[hy[j][2]]['score1']=1/(ratio+hy[j][3])
                except KeyError:
                    dictTem[hy[j][2]]={}
                    dictTem[hy[j][2]]['score1']=1/(ratio+hy[j][3])
        for j in range(len(ori)):
            if dl19_topics[i][0]==ori[j][0]:
                try:
                    dictTem[ori[j][2]]['score3']=1/(ratio+ori[j][3])
                except KeyError:
                    dictTem[ori[j][2]]={}
                    dictTem[ori[j][2]]['score3']=1/(ratio+ori[j][3])  
        for passage in dictTem.keys():
            num=len(dictTem[passage])/10+1
            score_sum=0
            for scores in dictTem[passage].keys():
                score_sum+=dictTem[passage][scores]
            dictTem[passage]=num*score_sum
        rank=0
        for key, value in sorted(dictTem.items(), key=lambda item: item[1],reverse=True):   
            rank+=1
            if rank<=1000:
                passageinfo=[dl19_hy[i][0],'Q0',key,rank,value,'rank']
                topicnum=dl19_topics[i][0]
                f.write(f'{topicnum} Q0 {key} {rank} {value} rank\n')

!python -m pyserini.eval.trec_eval -c -l 2 -m map dl19-passage dl19_splade_ed_Exp4Fuse
!python -m pyserini.eval.trec_eval -c -m ndcg_cut.10 dl19-passage dl19_splade_ed_Exp4Fuse
!python -m pyserini.eval.trec_eval -c -l 2 -m recall.1000 dl19-passage dl19_splade_ed_Exp4Fuse
   

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


/root/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar already exists!
Skipping download.
Running command: ['java', '-jar', '/root/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar', '-c', '-l', '2', '-m', 'map', '/root/.cache/pyserini/topics-and-qrels/qrels.dl19-passage.txt', 'dl19_splade_ed_Exp4Fuse']
Results:
map                   	all	0.5666


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


/root/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar already exists!
Skipping download.
Running command: ['java', '-jar', '/root/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar', '-c', '-m', 'ndcg_cut.10', '/root/.cache/pyserini/topics-and-qrels/qrels.dl19-passage.txt', 'dl19_splade_ed_Exp4Fuse']
Results:
ndcg_cut_10           	all	0.7753


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


/root/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar already exists!
Skipping download.
Running command: ['java', '-jar', '/root/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar', '-c', '-l', '2', '-m', 'recall.1000', '/root/.cache/pyserini/topics-and-qrels/qrels.dl19-passage.txt', 'dl19_splade_ed_Exp4Fuse']
Results:
recall_1000           	all	0.9291


In [10]:
#5.0 splade_sd

searcher = LuceneImpactSearcher('autodl-tmp/msmarco-v1-passage.splade-pp-sd','autodl-tmp/splade-cocondenser-selfdistil')

with open('dl19_splade_sd', 'w')  as f:
    for i in range(len(dl19_topics)):
        qid=dl19_topics[i][0]
        question=dl19_topics[i][1]
        hits = searcher.search(question, k=1000)
        rank = 0
        for hit in hits:
            rank += 1
            f.write(f'{qid} Q0 {hit.docid} {rank} {hit.score} rank\n')

!python -m pyserini.eval.trec_eval -c -l 2 -m map dl19-passage dl19_splade_sd
!python -m pyserini.eval.trec_eval -c -m ndcg_cut.10 dl19-passage dl19_splade_sd
!python -m pyserini.eval.trec_eval -c -l 2 -m recall.1000 dl19-passage dl19_splade_sd


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


/root/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar already exists!
Skipping download.
Running command: ['java', '-jar', '/root/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar', '-c', '-l', '2', '-m', 'map', '/root/.cache/pyserini/topics-and-qrels/qrels.dl19-passage.txt', 'dl19_splade_sd']
Results:
map                   	all	0.4998


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


/root/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar already exists!
Skipping download.
Running command: ['java', '-jar', '/root/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar', '-c', '-m', 'ndcg_cut.10', '/root/.cache/pyserini/topics-and-qrels/qrels.dl19-passage.txt', 'dl19_splade_sd']
Results:
ndcg_cut_10           	all	0.7358


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


/root/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar already exists!
Skipping download.
Running command: ['java', '-jar', '/root/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar', '-c', '-l', '2', '-m', 'recall.1000', '/root/.cache/pyserini/topics-and-qrels/qrels.dl19-passage.txt', 'dl19_splade_sd']
Results:
recall_1000           	all	0.8761


In [11]:
#5.1 splade_sd+Exp4Fuse

searcher = LuceneImpactSearcher('autodl-tmp/msmarco-v1-passage.splade-pp-sd','autodl-tmp/splade-cocondenser-selfdistil')

with open('dl19_splade_sd_hy', 'w')  as f:
    for i in range(len(dl19_hy)):
        qid=dl19_hy[i][0]
        question=dl19_hy[i][1]+'.'
        question=question*5
        question=question+dl19_hy[i][2]
        hits = searcher.search(question, k=1000)
        rank = 0
        for hit in hits:
            rank += 1
            f.write(f'{qid} Q0 {hit.docid} {rank} {hit.score} rank\n')
    
with open('dl19_splade_sd_hy', 'r') as file:
    hy1 = file.readlines()    
hy=[[int(x.split()[0]),x.split()[1],int(x.split()[2]),int(x.split()[3]),x.split()[4],x.split()[5]] for x in hy1]

with open('dl19_splade_sd', 'r') as file:
    ori1 = file.readlines()
ori=[[int(x.split()[0]),x.split()[1],int(x.split()[2]),int(x.split()[3]),x.split()[4],x.split()[5]] for x in ori1]

with open('dl19_splade_sd_Exp4Fuse', 'w')  as f:
    for i in range(len(dl19_topics)):
        dictTem={}
        for j in range(len(hy)):
            if dl19_topics[i][0]==hy[j][0]:
                try:
                    dictTem[hy[j][2]]['score1']=1/(ratio+hy[j][3])
                except KeyError:
                    dictTem[hy[j][2]]={}
                    dictTem[hy[j][2]]['score1']=1/(ratio+hy[j][3])             
        for j in range(len(ori)):
            if dl19_topics[i][0]==ori[j][0]:
                try:
                    dictTem[ori[j][2]]['score3']=1/(ratio+ori[j][3])
                except KeyError:
                    dictTem[ori[j][2]]={}
                    dictTem[ori[j][2]]['score3']=1/(ratio+ori[j][3])  
        for passage in dictTem.keys():
            num=len(dictTem[passage])/10+1
            score_sum=0
            for scores in dictTem[passage].keys():
                score_sum+=dictTem[passage][scores]
            dictTem[passage]=num*score_sum
        rank=0
        for key, value in sorted(dictTem.items(), key=lambda item: item[1],reverse=True):   
            rank+=1
            if rank<=1000: 
                topicnum=dl19_topics[i][0]
                f.write(f'{topicnum} Q0 {key} {rank} {value} rank\n')

!python -m pyserini.eval.trec_eval -c -l 2 -m map dl19-passage dl19_splade_sd_Exp4Fuse
!python -m pyserini.eval.trec_eval -c -m ndcg_cut.10 dl19-passage dl19_splade_sd_Exp4Fuse
!python -m pyserini.eval.trec_eval -c -l 2 -m recall.1000 dl19-passage dl19_splade_sd_Exp4Fuse
   

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


/root/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar already exists!
Skipping download.
Running command: ['java', '-jar', '/root/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar', '-c', '-l', '2', '-m', 'map', '/root/.cache/pyserini/topics-and-qrels/qrels.dl19-passage.txt', 'dl19_splade_sd_Exp4Fuse']
Results:
map                   	all	0.5657


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


/root/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar already exists!
Skipping download.
Running command: ['java', '-jar', '/root/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar', '-c', '-m', 'ndcg_cut.10', '/root/.cache/pyserini/topics-and-qrels/qrels.dl19-passage.txt', 'dl19_splade_sd_Exp4Fuse']
Results:
ndcg_cut_10           	all	0.7710


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


/root/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar already exists!
Skipping download.
Running command: ['java', '-jar', '/root/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar', '-c', '-l', '2', '-m', 'recall.1000', '/root/.cache/pyserini/topics-and-qrels/qrels.dl19-passage.txt', 'dl19_splade_sd_Exp4Fuse']
Results:
recall_1000           	all	0.9394
